In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from utils import * 

# For widescreen display, overrides utils.py settings
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Regression --> Predicting Outage Duration

In [3]:
# Load cleaned dataset
file_path = 'data/cleaned_outage_dataset.csv'
df = pd.read_csv(file_path)
df.head()

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,MONTH.NAME
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,2011-07-03,20:00:00,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,2.33e+06,2.11e+06,2.11e+06,6.56e+06,35.55,32.23,32.20,2.31e+06,276286.0,10673.0,2.60e+06,88.94,10.64,0.41,51268.0,47586.0,1.08,1.6,4802.0,274182.0,1.75,2.2,5348119,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2011-07-01 17:00:00,2011-07-03 20:00:00,Jul
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,2014-05-11,18:39:00,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,1.59e+06,1.81e+06,1.89e+06,5.28e+06,30.03,34.21,35.73,2.35e+06,284978.0,9898.0,2.64e+06,88.83,10.79,0.37,53499.0,49091.0,1.09,1.9,5226.0,291955.0,1.79,2.2,5457125,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2014-05-11 18:38:00,2014-05-11 18:39:00,May
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,2010-10-28,22:00:00,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,1.47e+06,1.80e+06,1.95e+06,5.22e+06,28.10,34.50,37.37,2.30e+06,276463.0,10150.0,2.59e+06,88.92,10.69,0.39,50447.0,47287.0,1.07,2.7,4571.0,267895.0,1.71,2.1,5310903,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2010-10-26 20:00:00,2010-10-28 22:00:00,Oct
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,2012-06-20,23:00:00,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,1.85e+06,1.94e+06,1.99e+06,5.79e+06,31.99,33.54,34.44,2.32e+06,278466.0,11010.0,2.61e+06,88.90,10.68,0.42,51598.0,48156.0,1.07,0.6,5364.0,277627.0,1.93,2.2,5380443,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2012-06-19 04:30:00,2012-06-20 23:00:00,Jun
4,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,2015-07-19,07:00:00,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,2.03e+06,2.16e+06,1.78e+06,5.97e+06,33.98,36.21,29.78,2.37e+06,289044.0,9812.0,2.67e+06,88.82,10.81,0.37,54431.0,49844.0,1.09,1.7,4873.0,292023.0,1.67,2.2,5489594,73.27,15.28,2279.0,1700.5,18.2,2.14,0.6,91.59,8.41,5.48,2015-07-18 02:00:00,2015-07-19 07:00:00,Jul


## Heavy skew in outage durations, most last less than 2 days

In [15]:
# Plot outage duration distribution
fig = px.histogram(df, x='OUTAGE.DURATION', nbins=100, title='Distribution of Outage Durations (Minutes)')
fig.update_layout(xaxis_title='Outage Duration (Minutes)', yaxis_title='Count')
fig.show()

# Plot log outage duration distribution
fig = px.histogram(df, x=np.log1p(df['OUTAGE.DURATION']), nbins=30, title='Distribution of Log Outage Durations (Minutes)')
fig.update_layout(xaxis_title='Log Outage Duration (Minutes)', yaxis_title='Count')
fig.show()